In [132]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from scipy.optimize import curve_fit

In [133]:
mp_key = "txVH20lqfEIM4Yxb2k5ptbih3BpgcIsN"
mp_id = "mp-8175"

In [134]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Chaleur spécifique

In [135]:
temperatures = np.arange(0,1000,5)
R = 8.314
nat = len(prim_struc) #nbr d'atome dans la cellule primitive
h_bar = 1.0545718e-34
kb = 1.380649e-23

#chaleurs spécifiques expérimentales
ph_cv = np.array([ph_dos.cv(temperatures[i]) for i in range(len(temperatures))])/(3*nat*R)

## Modèle d'Einstein

In [136]:
def einstein_model(T,Te):
    with np.errstate(over='ignore', divide='ignore', invalid='ignore'):
        x = np.where(T == 0, 0, Te / T)  # Remplace inf par 0 pour éviter erreurs
        exp_x = np.exp(x)
        cv = np.where(T == 0, 0, 3 * nat * R * (x**2 * exp_x) / (exp_x - 1)**2)
    return cv/(3*nat*R)

In [137]:
popt_einstein,_ = curve_fit(einstein_model, temperatures, ph_cv, p0=[300])

cv_einstein=einstein_model(temperatures, *popt_einstein)
print(f"Température d'Einstein optimale : {popt_einstein[0]:.2f} K")

Température d'Einstein optimale : 300.66 K


Pour obtenir le modèle d'Einstein on suppose que tous les atomes vibrent à une même fréquence $\omega_E$. On obtient alors pour la chaleur spécifique la formule suivante : $C_v = 3N_{at}R (\frac{\theta_E}{T})^2\frac{e^{\theta_E/T}}{(e^{\theta_E/T} - 1)^2} $

On y retrouve $N_{at}$ qui représente le nombre d'atome dans la cellule primitive et $\theta_E$, la température d'Einstein, qui est liée à la fréquence naturelle ($\omega_E$) par  $ k_B\theta_E = \hbar \omega_E$.

Mathématiquement on voit que si T >> $\theta_E$ (à haute temérature) on aura une chaleur spécifique qui tendra vers $3N_{at}R$. Dans le cas contraire, si T << $\theta_E$, $C_v$ sera proportionnel à $e^{-\theta_E/T}$.


La température d'Einstein obtenue par ajustement du modèle est 300.66K.

## Modèle de Debye

In [138]:
from scipy.integrate import quad

def debye_integrand(x):
    with np.errstate(over='ignore', divide='ignore', invalid='ignore'):
        exp_x = np.exp(x)
        safe_exp_x = np.where(exp_x == np.inf, 1e100, exp_x) 
        return (x**4 * safe_exp_x) / (safe_exp_x - 1)**2

# Modèle de Debye
def debye_model(T, Td):
    cv = np.zeros_like(T, dtype=float)
    for i, temp in enumerate(T):
        if temp == 0 or Td == 0:
            cv[i] = 0
        else:
            upper_limit = min(Td / temp, 50)  # Limite plus basse pour éviter les erreurs
            try:
                integral, _ = quad(debye_integrand, 0, upper_limit, limit=50)
            except:
                integral = 0  # En cas d'erreur d'intégration, retour à 0
            cv[i] = 9 * nat * R * (temp / Td)**3 * integral
    return cv/(3*nat*R)


In [139]:
popt_debye, _ = curve_fit(debye_model, temperatures, ph_cv,p0=[300])

cv_debye = debye_model(temperatures, *popt_debye)
print(f"Température de Debye optimale : {popt_debye[0]:.2f} K")

Température de Debye optimale : 407.75 K


Pour obtenir le modèle de Debye on suppose que toutes les fréquences de vibrations atomiques ont la même dispersion linéaire. On obtient alors pour la chaleur spécifique la formule suivante : $C_v = 9N_{at}R (\frac{T}{\theta_D})^3\int_0^{\theta_D/T} \frac{x^4 e^x}{(e^x - 1)^2} \, \mathrm{d}x $ 

On y retrouve $N_{at}$ qui représente le nombre d'atome dans la cellule primitive, $ x= \frac{\hbar\nu q}{k_BT}$ qui est une simplification pour résoudre l'intégrale et $\theta_D$ qui est la température de Debye et qui est liée à la fréquence de Debye ($\omega_D$) par  $ k_B\theta_D = \hbar \omega_D$.

Mathématiquement on voit que si T >> $\theta_D$ on aura une chaleur spécifique qui tendra vers $3N_{at}R$. Dans le cas contraire, si T << $\theta_D$, $C_v$ sera proportionnel à $T^3$. Cela reflète une contribution dominante des modes acoustiques de basse fréquence.

La température de Debye optimale obtenue par ajustement du modèle est 407.75K.\
Plus $\theta_D$ est important plus les liaisons interatomiques sont rigides et donc plus les phonons vibrent à haute fréquence. C'est assez typique que $\theta_D$ soit plus élevée que $\theta_E$, cela reflète que le modèle de Debye répartit l'énergie vibratoire sur un spectre plus large et tens vers une fréquence moyenne plus élevée que celle du modèle d'Einstein.

## Représentation des courbes de chaleur spécifique

In [140]:
fig = go.Figure()
scatter_exp = go.Scatter(x=temperatures, y=ph_cv, mode='lines', name='Expérimental',line=dict(color='green'))
scatter_debye =  go.Scatter(x=temperatures, y=cv_debye, mode='lines', name='Debye',line=dict(color='blue'))
scatter_einstein =  go.Scatter(x=temperatures, y=cv_einstein, mode='lines', name='Einstein',line = dict(color='red'))
fig.add_trace(scatter_exp)
fig.add_trace(scatter_debye)
fig.add_trace(scatter_einstein)
fig.add_hline(y=1, line_width=2, line_color="yellow", line_dash = 'dash')
fig.update_layout(
    xaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10, 'range':[0,1.2]},
    xaxis_title = "Temperature",
    yaxis_title = "C<sub>v</sub> / 3N<sub>at</sub>R",  
    title='Comparaison des chaleurs spécifiques')

fig.show()

Etant donné que l'on normalise nos valeurs de $C_v$ (divisées par $3N_{at}R$) il est logique de voir qu'à haute température (T>> $\theta_D$ et $\theta_E$)toutes les courbes $C_v$ tendent vers 1.\
Le profil des courbes issues de Debye et Einstein semble cohérent par rapport aux conclusions mathématiques faites précédemment. On voit également que le modèle de Debye est légèrement plus proche des valeurs expérimentales que celui d'Einstein. Cela peut s'expliquer par le fait que le modèle d4einstein repose sur une hypothèse plus forte, ce qui le rend moins rélaiste et donc plus éloignée de la réalité. 

On peut donc en conclure que le modèle de Debye fournit une description plus fidèle du comportement thermique du matériau, en particulier dans le régime à basse température cela le rend donc valide pour des approximations thermodynamiques.Le modèle d'Einstein, quant à lui, est correct pour des analyses simplifiées mais reste une approximation grossière dont la validitée est limitée aux hautes températures.

# Densité d'états de KTlO2

In [141]:
# valeurs expérimentales

ph_w=ph_dos.frequencies
ph_dos_exp=ph_dos.densities

In [142]:
def einstein_dos(f, fe):
    delta_f = (max(ph_w)-min(ph_w))/len(ph_w)
    return np.exp(-((f-fe)**2)/(2*(delta_f**2)))/ (np.sqrt(2*np.pi)*delta_f)

def debye_dos(f,fmax):
    return np.where(f>fmax,0,(f**2/fmax**3))


fdm = max(ph_w)*(popt_debye[0]/max(temperatures))
fe= max(ph_w)*(popt_einstein[0]/max(temperatures))

dos_debye = debye_dos(ph_w, fdm)
dos_einstein = einstein_dos(ph_w,fe)



La densité d'états phononiques g($\omega$) décrit le nombre de modes de vibrations qui existent à une fréquence donnée $\omega$. Elle est aussi appelée densité de modes normaux. Une fois normalisée, elle satisfait :  $\int g(\omega) \mathrm{d}\omega = 3N_{at}$


Pour $\textbf{Einstein}$ : toujours sous la même hypothèse que tous les atomes vibrent à une même fréquence $\omega_E$ (notée fe dans le code) : $g_E(\omega) = 3N_{at}\delta(\omega-\omega_E)$.\
Cette fonction est un delta de Dirac avec son pic en $\omega_E$. Notre code trace une courbe centrée sur la fréquence d'Einstein avec une très faible épaisseur autour de $\omega_E$ afin de représenter au mieux le $\delta$. On calcule ensuite fe (qui est $\omega_E$) par rapport à la température d'Einstein optimale trouvée.

Pour $\textbf{Debye}$ : l'hypothèse reste que toutes les fréquence varient linéairement de la même manière jusqu'à une certaine fréquence $\omega_D$: $ g_D(\omega) = \begin{cases}\frac{3N_{at}\Omega}{2\pi^2} \cdot \frac{\omega^2}{\nu^3} & \text{si } \omega \leq \omega_D = \nu q_D \text{ ($\nu$ est la vitesse du son)} \\\\ 0 & \text{si } \omega > \omega_D \end{cases}$.\
On voit donc que pour toute fréquence $\leq \omega_D$ la densité d'état sera proportionnelle à $\frac{\omega^2}{\nu^3}$. Cela signifie que à haute fréquence les phonons sont moins nombreux qu'à basse fréquence. Pour les fréquences au delà de $\omega_D$ la densité d'états est nulle, il n'y a pas de phonons.\
Dans notre code f représente la fréquence $\omega$ et fmax (fdm calculé à partir de la température optimale de Debye) est la fréquence de Debye qui est directement lié à $\nu$ (présent dans la formule) pas la relation $\omega_D = \nu q$. Notre code utilise en fait la forme simplifiée : $g_D(\omega) = \begin{cases}\frac{\omega^2}{\omega_D^3} & \text{si } \omega \leq \omega_D  \\\\ 0 & \text{si } \omega > \omega_D \end{cases}$


In [146]:
fig_dos = go.Figure()

scatter_dos_exp = go.Scatter(x=ph_w, y=ph_dos_exp, mode='lines', name='Expérimental', line=dict(color='green'))
scatter_dos_debye = go.Scatter(x=ph_w, y=dos_debye, mode='lines', name='Debye',line = dict(color='blue'))
scatter_dos_einstein = go.Scatter(x=ph_w, y=dos_einstein, mode='lines', name='Einstein', line=dict(color='red'))

fig_dos.add_trace(scatter_dos_exp)
fig_dos.add_trace(scatter_dos_debye)
fig_dos.add_trace(scatter_dos_einstein)

fig_dos.update_layout(
    xaxis={'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen': 10},
    yaxis={'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen': 10, 'range' : [0,4]},
    xaxis_title="Fréquence (THz)",
    yaxis_title="Densité d'états phononique",
    title="Comparaison des densités d’états phononiques"
)

fig_dos.show()

On observe bien un pic de densité (représentation du Delta de Dirac) dans la courbe d'Einstein. La courbe de Debye suit une loi quadratique et s'annule ensuite. On sait que la position du pic sur la courbe d'Einstein et de la coupure de la courbe de Debye correspondent respectivement à la fréquence $\omega_E$ et $\omega_D$. Etant donné que ces fréquences sont proportionnelles à la température et que nous avons observé que $\theta_E$ était plus faible que $\theta_D$, il est logique d'observer que $\omega_E < \omega_D$.

In [147]:
fig_ph_bs_and_dos = get_plot_bs_and_dos(ph_bs, ph_dos)
fig_ph_bs_and_dos.add_trace(go.Scatter(x=dos_debye, y=ph_w, mode='lines', name='Debye',
                                       line=dict(dash='dash', color='blue')),row=1,col=2)

fig_ph_bs_and_dos.add_trace(go.Scatter(x=dos_einstein, y=ph_w, mode='lines', name='Einstein',
                                       line=dict(dash='dot', color='red')),row=1, col=2)
                        
fig_ph_bs_and_dos.update_yaxes(rangemode="tozero")
fig_ph_bs_and_dos.show()

Tout d'abord on observe que la densité d'état expérimentale est nulle dans la bande interdite, cela reflète l'absence de vibration (mode phononique) dans cette plage de fréquence.

La courbe de Debye est nulle dans la région des bandes optiques (à hautes fréquences), cela est normal car le modèle suppose que tous les modes sont acoustiques, il ne considère que les vibrations collectives.

On observe que les deux modèle ne peuvent pas reproduire l'existence de bandes interdites car ils ne prennent pas en compte la structure de bande réelle du spectre phononique à cause de leurs hypothèses très simplificatrices.
